# Calibrazione della camera

In questo foglio affronteremo uno dei passaggi cardine quando si lavora con una camera: la correzione degli distorsioni!

Bisogna infatti sapere che la maggior parte delle fotocamere introduce una significativa distorsione alle immagini che cattura, le due più importanti sono la distorsione radiale e quella tangenziale. La prima causa alle linee rette di apparire curve e aumenta man mano che ci si allontana dal centro dell'immagine, mentre la seconda è data dal fatto che il piano della lente non è parallelo al piano dell'immagine.

![esempio radiale]("part1/img/calib_radial.jpg")
![esempio tangenziale]("part1/img/tangential_distortion.png")

La procedura che va a calcolare i parametri relativi a questi errori per darci una matrice di correzione è detta **calibrazione intrinseca**

Per questo corso ti servirà anche stampare la scacchiera che troverai nel materiale di questo modulo. Questa risulta uno strumento perfetto in quando ci fornisce dei punti facilmente individuabili e dalle coordinate relative ben definite. Sarà inoltre importante che questa rimanga ben piatta, consigliamo quindi di incollarla ad una superficie solida (ex. cartoncino duro).

Cominciamo a programmare, importiamo innanzitutto le librerie necessarie:

In [ ]:
import cv2
import glob
import numpy as np
import yaml
import time

Andiamo ora a creare delle variabili che rappresentano i dati della nostra scacchiera, in particolare nx ed ny non rappresenteranno il numero di caselle ma il numero di spigoli interni alla scacchiera. Lo square_size, ovvero la dimensione del lato dei quadrati della scacchiera dovrà essere in metri.
TO-DO: aggiungere immagine spiegazione

In [ ]:
nx = 9
ny = 7
square_size = 0.018

andiamo ora a creare gli array che conserveranno le coordinate dei punti di interesse della scacchiera. imgpoint conterrà le coordinate dei punti rilevati nelle foto, mentre objpoints conserverà i punti nello spazio 3D e le loro coordinate saranno molto semplici dato che consideriamo la scacchiera piana e a terra (Z = 0), quindi avremo punti del tipo (0,0,0), (0.018,0,0),...

In [ ]:
objp = np.zeros((nx * ny, 3), np.float32)
objp[:, :2] = np.mgrid[0:self.nx, 0:self.ny].T.reshape(-1, 2)
objpoints = objp * square_size

imgpoints = []

andiamo ora ad importare le foto scattate per la calibrazione. Se si vuole ottenere un'effettivo file di calibrazione della propria telecamera sarebbe bene scattarsele da solo. Alla fine di questo modulo si troverà un programma per poter effettuare questo lavoro.

In [ ]:
images = glob.glob('part1/chessboards/*.jpg')

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)

    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

In [ ]:
ret, matrix, distortion_coeff, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

e ora andiamo a salvare le matrici in un file yaml, questo ci permetterà di tenerle salvate per futuri utilizzi.

In [ ]:
calibration_data = {
    "camera_matrix": matrix,
    "distortion_coeff": distortion_coeff
}

with open('part1/calibration.yml', 'w') as outfile:
    yaml.dump(calibration_data, outfile, default_flow_style = False)

Questo file di calibrazione ci servirà per tutti i futuri moduli! Finora però avete utilizzato foto fornite da noi, ma se voleste effettuare una calibrazione precisa sarebbe meglio che utilizzaste delle vostre foto, e ora vi spiegherò come.

Prendete la scacchiera che avete stampato e posizionatela davanti alla camera, cambiando posizione ed angolazione ad ogni foto fatta. Assicuratevi comunque che questa si veda bene nella preview.

andremo inoltre ad importare una libreria necessaria per l'utilizzo della camera per chi sta seguendo questo tutorial da raspberry con una picamera

In [ ]:
from picamera import PiCamera

In [ ]:
Ora andremo a runnare il programma che scatterà le foto, questa lascierà qualche secondo di preview per permettere il cambio di posizione della schacchiera.
Scatteremo 15 foto in modo da aumentare il numero di possibili foto buone. Per chi stesse usando una picamera la casella è la prima, mentre per chi stesse usando una camera USB la seconda

In [ ]:
camera = Picamera()
for i in range(10):
    print("avvio camera")
    camera.start_preview()
    print("foto tra 4 secondi")
    time.sleep(4)
    photo_name = "part1/foto/foto" + i + ".jpg"
    camera.capture(photo_name)
    i=i+1
    camera.stop_preview()

Il programma per chi usa una USB camera aspetterà che voi premiate s per salvare la foto; è possibile premere q per stopparlo in anticipo.

In [ ]:
cam = cv2.VideoCapture(0)
for i in range(10):
    print("avvio camera")
    ret, image = cam.read()
	cv2.imshow('photo',image)
	k = cv2.waitKey(1)
    if k == ord(s):
        photo_name = "part1/foto/foto" + i + ".jpg"
        cv2.imwrite(photo_name, image)
        i=i+1
    if k == ord(q):
		break

controlliamo ora quante foto buone sono state scattate.

In [ ]:
total = 0

images = glob.glob('part1/foto/*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)
    if ret == True:
        total = total + 1
print("numero di foto buone: {}".format(total))